Importing necessary libraries

In [1]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import classification_report, confusion_matrix

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Lemmatizer

In [29]:

lemmatizer = WordNetLemmatizer()

Loading intents from JSON file

In [30]:
with open('intents.json') as json_file:
    intents = json.load(json_file)

Data Preprocessing

In [31]:
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

Creating training data

In [32]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns if word.lower() not in ignore_letters]
    for word in words:
        bag.append(1 if word in word_patterns else 0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

Model

In [33]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))

# Compiling the model
optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

Model Training

In [34]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbotmodel.h5')
print('Training Done')

Epoch 1/200
26/26 [==============================] - 2s 7ms/step - loss: 3.1272 - accuracy: 0.0312
Epoch 2/200
26/26 [==============================] - 0s 7ms/step - loss: 3.0410 - accuracy: 0.0703
Epoch 3/200
26/26 [==============================] - 0s 9ms/step - loss: 3.0010 - accuracy: 0.1094
Epoch 4/200
26/26 [==============================] - 0s 8ms/step - loss: 2.9001 - accuracy: 0.1875
Epoch 5/200
26/26 [==============================] - 0s 8ms/step - loss: 2.8706 - accuracy: 0.1484
Epoch 6/200
26/26 [==============================] - 0s 6ms/step - loss: 2.6341 - accuracy: 0.1953
Epoch 7/200
26/26 [==============================] - 0s 5ms/step - loss: 2.4838 - accuracy: 0.3047
Epoch 8/200
26/26 [==============================] - 0s 6ms/step - loss: 2.3482 - accuracy: 0.2969
Epoch 9/200
26/26 [==============================] - 0s 7ms/step - loss: 2.1261 - accuracy: 0.3281
Epoch 10/200
26/26 [==============================] - 0s 6ms/step - loss: 2.1211 - accuracy: 0.3438
Epoch 11/

c:\Users\otaod\anaconda3\envs\ML\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model Evaluation

In [35]:
loss, accuracy = model.evaluate(np.array(train_x), np.array(train_y), verbose=0)
print(f'Training Accuracy: {accuracy:.2f}')
print(f'Training Loss: {loss:.2f}')

from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(np.array(train_x))
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(np.array(train_y), axis=1)

print("Classification Report:")
print(classification_report(y_true, y_pred_classes, target_names=classes))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))

Training Accuracy: 0.91
Training Loss: 0.17
4/4 [==============================] - 0s 3ms/step
Classification Report:
                        precision    recall  f1-score   support

             Allergies       1.00      1.00      1.00         2
       Asthma symptoms       1.00      0.83      0.91         6
          Consultation       1.00      1.00      1.00         5
   Depression symptoms       1.00      0.86      0.92         7
     Diabetes symptoms       0.86      1.00      0.92         6
  MedicineAvailability       1.00      1.00      1.00         6
                   age       1.00      1.00      1.00         5
   arthritis medicines       1.00      1.00      1.00         6
  arthritis prevention       1.00      1.00      1.00         5
    arthritis symptoms       1.00      0.38      0.55         8
     asthma prevention       1.00      1.00      1.00         4
common cold prevention       1.00      1.00      1.00         5
  common cold symptoms       1.00      0.50      